In [ ]:
import numpy as np
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt

# LMS Filter Configuration
filter_length = 8  
learning_rate = 0.016  
filter_weights = np.zeros(filter_length)
noise_buffer = np.zeros(filter_length)

def lms_filter_sample(reference_noise, noisy_signal):
    """
    Apply LMS filter to a single sample.
    
    Args:
        reference_noise: Reference noise sample
        noisy_signal: Noisy input signal sample
    
    Returns:
        tuple: (estimated_noise, estimated_clean_signal)
    """
    global filter_weights, noise_buffer, learning_rate

    # Update buffer with new reference noise sample
    noise_buffer[1:] = noise_buffer[:-1]
    noise_buffer[0] = reference_noise

    # Estimate noise using current filter weights
    estimated_noise = np.dot(filter_weights, noise_buffer)

    # Estimate clean signal
    estimated_clean = noisy_signal - estimated_noise

    # Update filter weights using LMS algorithm
    filter_weights += learning_rate * estimated_clean * noise_buffer

    return estimated_noise, estimated_clean

def add_noise(clean_audio, noise_type="white", snr_db=10):
    """
    Add noise to clean audio signal.
    
    Args:
        clean_audio: Clean audio signal
        noise_type: Type of noise ("white" or "brown")
        snr_db: Signal-to-noise ratio in dB
    
    Returns:
        tuple: (noisy_audio, noise)
    """
    if noise_type == "white":
        noise = np.random.randn(len(clean_audio))  
    elif noise_type == "brown":
        noise = np.cumsum(np.random.randn(len(clean_audio)))
        noise -= np.mean(noise)

    # Calculate scaling factor for desired SNR
    clean_power = np.mean(clean_audio**2)
    noise_power = np.mean(noise**2)
    scaling_factor = np.sqrt(clean_power / (noise_power * (10 ** (snr_db / 10))))
    scaled_noise = noise * scaling_factor

    return clean_audio + scaled_noise, scaled_noise

def process_audio_file(noisy_filename, noise_filename):
    """
    Process audio file using LMS filter.
    
    Args:
        noisy_filename: Path to noisy audio file
        noise_filename: Path to noise reference file
    
    Returns:
        dict: Processing results including filtered audio
    """
    global filter_weights, noise_buffer
    
    # Reset filter state
    filter_weights.fill(0)
    noise_buffer.fill(0)
    
    # Load audio files
    sample_rate, noisy_audio = wav.read(noisy_filename)

    # Convert to float
    if noisy_audio.dtype == np.int16:
        noisy_audio = noisy_audio.astype(np.float32) / 32768.0

    noise = np.load(noise_filename)

    # Initialize output arrays
    filtered_audio = np.zeros_like(noisy_audio)
    filter_output = np.zeros_like(noisy_audio)
    
    # Process each sample with the LMS filter
    for i in range(len(noisy_audio)):
        estimated_noise, estimated_clean = lms_filter_sample(noise[i], noisy_audio[i])
        filter_output[i] = estimated_noise
        filtered_audio[i] = estimated_clean
    
    # Save filtered audio
    filtered_filename = "LMS_denoised.wav" 
    filtered_audio_int = (filtered_audio * 32767).astype(np.int16)
    wav.write(filtered_filename, sample_rate, filtered_audio_int)
    
    print(f"\nOutput files created:")
    print(f"- {filtered_filename}")
    
    return {
        'noisy_audio': noisy_audio,
        'filtered_audio': filtered_audio,
        'noise': noise,
        'sample_rate': sample_rate
    }

def compare_performance(clean_filename, noisy_audio, filtered_audio):
    """
    Compare performance between original noisy audio and filtered audio.
    
    Args:
        clean_filename: Path to clean reference audio
        noisy_audio: Noisy audio signal
        filtered_audio: Filtered audio signal
    
    Returns:
        dict: Performance metrics including SNR improvements
    """
    sample_rate, clean_audio = wav.read(clean_filename)

    if clean_audio.dtype == np.int16:
        clean_audio = clean_audio.astype(np.float32) / 32768.0

    # Calculate noise power
    noise_power_original = np.mean((noisy_audio - clean_audio) ** 2)
    noise_power_filtered = np.mean((filtered_audio - clean_audio) ** 2)
    
    # Calculate SNR values
    signal_power = np.mean(clean_audio ** 2)
    snr_original = 10 * np.log10(signal_power / noise_power_original)
    snr_filtered = 10 * np.log10(signal_power / noise_power_filtered)
    improvement_db = snr_filtered - snr_original
    
    # Display results
    print(f"\nPerformance Metrics:")
    print(f"Original SNR: {snr_original:.2f} dB")
    print(f"Filtered SNR: {snr_filtered:.2f} dB")
    print(f"SNR Improvement: {improvement_db:.2f} dB")
    
    return {
        'clean_audio': clean_audio,
        'snr_original': snr_original,
        'snr_filtered': snr_filtered,
        'improvement_db': improvement_db
    }

if __name__ == "__main__":
    """
    Main execution flow for LMS audio filtering demonstration.
    """
    clean_filename = "mono_1_.wav"
    
    print("LMS Audio Filter Demo")
    print("=" * 30)
    print("Adding noise to clean audio...")
    
    # Load clean audio
    sample_rate, clean_audio = wav.read(clean_filename)
    if clean_audio.dtype == np.int16:
        clean_audio = clean_audio.astype(np.float32) / 32768.0

    # Add noise to create test scenario
    noisy_audio, noise = add_noise(clean_audio, noise_type="white", snr_db=10)

    # Save noisy audio and noise reference
    wav.write("LMS_filter_noisy.wav", sample_rate, (noisy_audio * 32767).astype(np.int16))
    np.save("LMS_filter_noise.npy", noise)
    print("Created noisy audio and noise reference files")

    # Process audio with LMS filter
    print("\nProcessing audio with LMS filter...")
    results = process_audio_file("LMS_filter_noisy.wav", "LMS_filter_noise.npy")
    filtered_audio = results['filtered_audio']
   
    # Compare performance
    print("\nEvaluating filter performance...")
    comparison = compare_performance(clean_filename, results['noisy_audio'], filtered_audio)
    results.update(comparison)

    print("\nDemo completed successfully!")

In [ ]:

#ploting the results
time = np.arange(len(clean_audio)) / sample_rate
plt.figure(figsize=(15, 10))
# Waveforms
plt.subplot(3, 2, 1)
plt.plot(time, clean_audio)
plt.title('Clean Audio - Waveform')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.subplot(3, 2, 3)
plt.plot(time, noisy_audio)
plt.title('Noisy Audio - Waveform')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.subplot(3, 2, 5)
plt.plot(time, filtered_audio)
plt.title('Filtered Audio - Waveform')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
# Spectrograms
plt.subplot(3, 2, 2)
plt.specgram(clean_audio, Fs=sample_rate, NFFT=1024, noverlap=512, cmap='viridis')
plt.title('Clean Audio - Spectrogram')
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.colorbar(format='%+2.0f dB')
plt.subplot(3, 2, 4)
plt.specgram(noisy_audio, Fs=sample_rate, NFFT=1024, noverlap=512, cmap='viridis')
plt.title('Noisy Audio - Spectrogram')
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.colorbar(format='%+2.0f dB')
plt.subplot(3, 2, 6)
plt.specgram(filtered_audio, Fs=sample_rate, NFFT=1024, noverlap=512, cmap='viridis')
plt.title('Filtered Audio - Spectrogram')
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.colorbar(format='%+2.0f dB')
plt.tight_layout()
plt.show()

